### Setting

In [1]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
WORK_DIR = os.environ.get("WORK_DIR")
print(WORK_DIR)
sys.path.append(WORK_DIR)

from openai import OpenAI
from notion_client import Client
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
NOTION_DB_ID = os.environ.get("NOTION_DB_ID")

/PaperSurvey


### Function

In [3]:
def create_markdown_page(notion, database_id, title, markdown_content):
    """
    マークダウンコンテンツを含むNotionページを作成する
    
    Parameters:
    - notion: Notion Client インスタンス
    - database_id: データベースID
    - title: ページタイトル
    - markdown_content: マークダウンコンテンツ
    """
    
    # マークダウンをNotionブロックに変換する関数
    def convert_heading_to_blocks(text, level):
        return {
            'object': 'block',
            'type': f'heading_{level}',
            f'heading_{level}': {
                'rich_text': [{'type': 'text', 'text': {'content': text}}]
            }
        }
    
    # コンテンツをブロックに変換
    blocks = []
    current_text = []
    
    for line in markdown_content.split('\n'):
        # 見出し1の処理 (#)
        if line.startswith('# '):
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            blocks.append(convert_heading_to_blocks(line[2:].strip(), 1))
            
        # 見出し2の処理 (##)
        elif line.startswith('## '):
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            blocks.append(convert_heading_to_blocks(line[3:].strip(), 2))
            
        # 空行の処理
        elif line.strip() == '':
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            
        # 通常のテキスト処理
        else:
            current_text.append(line)
    
    # 残りのテキストを処理
    if current_text:
        blocks.append({
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
            }
        })

    # ページの作成
    return notion.pages.create(
        **{
            'parent': {'database_id': database_id},
            'properties': {
                'title': {
                    'title': [
                        {
                            'text': {
                                'content': title
                            }
                        }
                    ]
                }
            },
            'children': blocks
        }
    )

### Impliment

In [4]:
import pymupdf4llm 

pdf_file = "/PaperSurvey/docs/prototype/BLT ByteLatentTransformer.pdf"
title = "BLT ByteLatentTransformer"
md_text = pymupdf4llm.to_markdown(pdf_file)

print(md_text)

Processing /PaperSurvey/docs/prototype/BLT ByteLatentTransformer.pdf...
[                                        ] (0/2[=                                       ] ( 1/2[==                                      ] ( 2/27=[====                                    ] ( 3/27[=====                                   ] ( 4/2=[=======                                 ] ( 5/2[========                                ] ( 6/27=[==========                              ] ( 7/27[===========                             ] ( 8/2=[=============                           ] ( 9/2[==============                          ] (10/27=[================                        ] (11/27[=================                       ] (12/2=[===================                     ] (13/2[====================                    ] (14/27=[======================                  ] (15/27[=======================                 ] (16/2=[=========================               ] (17/2[==========================              ] (18/27

In [5]:
import time

debug: bool = False
prompt_path = "/PaperSurvey/prompts/v1.txt"
client = OpenAI()

with open(prompt_path, encoding="UTF-8") as f:
    prompt = f.read()

prompt = prompt.replace("<<INPUT>>", md_text)

if debug:
    print(prompt)
    time.sleep(20)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
    {
        "role": "user",
        "content": prompt,
    }
    ],
    temperature=0,
    max_tokens=10000,
    top_p=0,
    frequency_penalty=None,
    presence_penalty=0.1,
)

In [6]:
md_response = response.choices[0].message.content
md_response

'```\n# abstract\n本論文では、Byte Latent Transformer (BLT)というトークナイザー不要のアーキテクチャを提案します。このアーキテクチャは、生のバイトデータから学習し、トークン化ベースのモデルと同等のパフォーマンスを実現し、効率性と堅牢性において大きな改善をもたらします。従来の大規模言語モデルは、トークン化というヒューリスティックな前処理ステップを除いてほぼエンドツーエンドで訓練されており、トークン化が文字列の圧縮方法にバイアスをかけるため、ドメインやモダリティの感受性、入力ノイズへの感受性、正字法の知識の欠如、多言語の不平等といった問題が生じています。\n\n# 解決する課題\n## 既存研究の流れ（関連研究）\n従来の大規模言語モデルは、トークン化を必要とし、バイトに直接訓練することはコストが高いため、より効率的な自己注意や注意不要のアーキテクチャが提案されていますが、これらは主に小規模モデルの訓練に役立つものでした。\n\n## この研究が解決する課題・どう解決するのか\n解決する課題1: トークン化の必要性を排除する\n\u3000→BLTは、バイトを動的にパッチにグループ化する学習可能な方法を提案し、固定語彙を持たないパッチ表現を使用します。\n\n解決する課題2: 計算資源の効率的な配分\n\u3000→BLTは、次のバイト予測のエントロピーに基づいてデータをセグメント化し、情報密度が比較的一様なバイトのコンテキスト化されたグループを作成します。\n\n解決する課題3: 入力ノイズへの堅牢性の向上\n\u3000→生のバイトデータを直接扱うことで、トークン化ベースのモデルよりもノイズの多い入力に対して堅牢性が向上します。\n\n# 提案手法\n## 提案手法の直感的な説明\nBLTは、バイトをパッチに動的にグループ化し、計算資源をデータの複雑さに基づいて効率的に配分する新しいアーキテクチャです。\n\n## 提案手法詳細\nBLTは、ローカルエンコーダ、グローバルラテントトランスフォーマー、ローカルデコーダの3つのモジュールで構成されます。ローカルエンコーダはバイトをパッチ表現に変換し、グローバルモデルはパッチ表現を処理し、ローカルデコーダはパッチから生のバイトを生成します。\n\n# 実験\n## 実験設定\n

In [7]:
cleaned_text = md_response.strip('`').strip()
cleaned_text

'# abstract\n本論文では、Byte Latent Transformer (BLT)というトークナイザー不要のアーキテクチャを提案します。このアーキテクチャは、生のバイトデータから学習し、トークン化ベースのモデルと同等のパフォーマンスを実現し、効率性と堅牢性において大きな改善をもたらします。従来の大規模言語モデルは、トークン化というヒューリスティックな前処理ステップを除いてほぼエンドツーエンドで訓練されており、トークン化が文字列の圧縮方法にバイアスをかけるため、ドメインやモダリティの感受性、入力ノイズへの感受性、正字法の知識の欠如、多言語の不平等といった問題が生じています。\n\n# 解決する課題\n## 既存研究の流れ（関連研究）\n従来の大規模言語モデルは、トークン化を必要とし、バイトに直接訓練することはコストが高いため、より効率的な自己注意や注意不要のアーキテクチャが提案されていますが、これらは主に小規模モデルの訓練に役立つものでした。\n\n## この研究が解決する課題・どう解決するのか\n解決する課題1: トークン化の必要性を排除する\n\u3000→BLTは、バイトを動的にパッチにグループ化する学習可能な方法を提案し、固定語彙を持たないパッチ表現を使用します。\n\n解決する課題2: 計算資源の効率的な配分\n\u3000→BLTは、次のバイト予測のエントロピーに基づいてデータをセグメント化し、情報密度が比較的一様なバイトのコンテキスト化されたグループを作成します。\n\n解決する課題3: 入力ノイズへの堅牢性の向上\n\u3000→生のバイトデータを直接扱うことで、トークン化ベースのモデルよりもノイズの多い入力に対して堅牢性が向上します。\n\n# 提案手法\n## 提案手法の直感的な説明\nBLTは、バイトをパッチに動的にグループ化し、計算資源をデータの複雑さに基づいて効率的に配分する新しいアーキテクチャです。\n\n## 提案手法詳細\nBLTは、ローカルエンコーダ、グローバルラテントトランスフォーマー、ローカルデコーダの3つのモジュールで構成されます。ローカルエンコーダはバイトをパッチ表現に変換し、グローバルモデルはパッチ表現を処理し、ローカルデコーダはパッチから生のバイトを生成します。\n\n# 実験\n## 実験設定\nBLTモデ

In [9]:
notion_client = Client(auth=NOTION_TOKEN)
page = create_markdown_page(notion_client, NOTION_DB_ID, title, cleaned_text)

In [10]:
response.usage

CompletionUsage(completion_tokens=1076, prompt_tokens=23948, total_tokens=25024, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [12]:
print(150 * 0.15 * response.usage.prompt_tokens / 1000000)
print(150 * 0.6 *  response.usage.completion_tokens / 1000000)

0.53883
0.09684


### MeshXL

In [13]:
pdf_file = "/PaperSurvey/docs/prototype/MeshXL.pdf"
title = "MeshXL"
md_text = pymupdf4llm.to_markdown(pdf_file)

prompt_path = "/PaperSurvey/prompts/v1.txt"
client = OpenAI()

with open(prompt_path, encoding="UTF-8") as f:
    prompt = f.read()

prompt = prompt.replace("<<INPUT>>", md_text)

if debug:
    print(prompt)
    time.sleep(20)

response1 = client.chat.completions.create(
    model="gpt-4o",
    messages=[
    {
        "role": "user",
        "content": prompt,
    }
    ],
    temperature=0,
    max_tokens=10000,
    top_p=0,
    frequency_penalty=None,
    presence_penalty=0.1,
)

cleaned_text1 = response1.choices[0].message.content.strip('`').strip()
create_markdown_page(notion_client, NOTION_DB_ID, title, cleaned_text1)

print(150 * 0.15 * response1.usage.prompt_tokens / 1000000)
print(150 * 0.6 *  response1.usage.completion_tokens / 1000000)

Processing /PaperSurvey/docs/prototype/MeshXL.pdf...
[                                        ] (0/1=[==                                      ] ( 1/18=[====                                    ] ( 2/18=[======                                  ] ( 3/18=[========                                ] ( 4/18==[===========                             ] ( 5/1=[=============                           ] ( 6/1=[===============                         ] ( 7/1=[=================                       ] ( 8/1==[====================                    ] ( 9/18=[======================                  ] (10/18=[========================                ] (11/18=[==========================              ] (12/18=[============================            ] (13/18==[===============================         ] (14/1=[=================================       ] (15/1=[===================================     ] (16/1=[=====================================   ] (17/1==[========================================] (18/18]
0.